In [5]:
import numpy as np
import matplotlib.pyplot as plt

def Markov(X, nbin=100, lags=range(1, 100), initial_scales=[1, 10], X_scale='linear', Y_scale='linear'):
    """
    Estimating Markov-Einstein Length using Wilcoxon test as described in: 
    "Different methods to estimate the Einstein-Markov coherence length in turbulence", Stresing et al. PRE 83, 046319 (2011).
    
    Parameters:
    X (array-like)         : Input data to analyze.
    nbin (int, optional)   : Number of bins for discretization of X. Default is 100.
    lags (iterable)        : List of lags to use for the analysis. Default is range(1, 30). 
    initial_scales (list)  : List of initial scales, "t0", to consider. Default is [1, 10].
    X_scale (str, optional): Scale for the x-axis ('linear' or 'log'). Default is 'linear'.
    Y_scale (str, optional): Scale for the y-axis ('linear' or 'log'). Default is 'linear'.
    
    Returns:
    None: Displays a plot of the average Delta_Q values against the lag values.
    
    Raises:
    ValueError: If the number of data points is insufficient.
    """
    
    # Discretize the input data X
    disc = np.digitize(X, np.linspace(np.min(X), np.max(X), nbin))
    N = len(X)
    
    # Iterate over initial scales
    for t1 in initial_scales:
        
        M = N - t1 - 2 * max(lags)
        
        if M <= 0:
            raise ValueError("Number of data points must be much larger than max(initial_scales) + 2 * max(lags)")
        
        v1 = disc[t1:M + t1]
        
        O = 0
        Delta_Q_avg = np.zeros(len(lags))
        
        for tau in lags:
            t2 = t1 + tau
            v2 = disc[t2:M + t2]
            t3 = t2 + tau
            v3 = disc[t3:M + t3]
            
            Delta_Q = []
            
            for V in np.unique(v2):
                cond1 = np.where(v2 == V)[0]
                xx = np.sort(v1[cond1])
                n = len(xx)
                cond2 = np.where(v3 == np.median(v3))[0]
                cond = np.intersect1d(cond1, cond2)
                if len(cond) > 0:
                    yy = np.sort(v1[cond])
                    m = len(yy)
                    # Ensure n * m > 25 for the calculation
                    if n * m > 25:
                        # Number of inversions for y[i] = q_y[i]
                        q_y = np.zeros(m)
                        for i in range(m):
                            q_y[i] = np.sum(xx < yy[i])
                        Q = np.sum(q_y)
                        
                        Q_mean = m * n / 2
                        Q_sd = np.sqrt(n * m * (n + m + 1) / 12)
                        
                        # If Q < sqrt(2/pi), then Markov
                        Delta_Q.append(np.abs(Q - Q_mean) / (Q_sd * np.sqrt(2 / np.pi)))
            
            if len(Delta_Q) == 0:
                Delta_Q_avg[O] = np.nan
            else:
                Delta_Q_avg[O] = np.mean(Delta_Q)
            
            O += 1
        
        # Plot the results for the current initial scale
        plt.plot(lags, Delta_Q_avg, marker='o', linestyle='', label=f't0 = {t1}')
    
    # Customize the plot
    plt.legend()
    plt.xlabel('Lags')
    plt.ylabel('<Delta_Q>')
    plt.xscale(X_scale)
    plt.yscale(Y_scale)
    plt.title('Wilcoxon Test for Markovianity')
    plt.show()

# Example usage:
# X = np.random.rand(10000)  # Replace with your dataset
# Markov(X)